In [ ]:
import netket as nk
import json
import matplotlib.pyplot as plt
import numpy as np
import jax
import qutip as qt
import netket.experimental as nkx
import optax as opt

# Initialization part

In [ ]:
# definie the parameters of the model
L = 5
alpha = 0.9
V = 2.0
gamma = 1.0

# define hilbert space and hamiltonian dimensions
g = nk.graph.Hypercube(length=L, n_dim=1, pbc=True)
h_space = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
hamiltonian = nk.operator.LocalOperator(h_space)

## create storage for the observables and jumps operators
j_ops = []
obs_sx = nk.operator.LocalOperator(h_space)
obs_sy = nk.operator.LocalOperator(h_space, dtype=complex)
obs_sz = nk.operator.LocalOperator(h_space)

# build hamiltonian and observables
for i in range(L):
    hamiltonian += alpha/2 * nk.operator.spin.sigmax(h_space, i)
    hamiltonian += (V/4 * nk.operator.spin.sigmaz(h_space, i)
                    * nk.operator.spin.sigmaz(h_space, (i + 1) % L)
    )
    obs_sx += nk.operator.spin.sigmax(h_space, i) / L
    obs_sy += nk.operator.spin.sigmay(h_space, i)/ L
    obs_sz += nk.operator.spin.sigmaz(h_space, i)/ L

    
    j_ops.append(gamma *nk.operator.spin.sigmam(h_space, i))

# create the Liouvillian superoperator
lind = nk.operator.LocalLiouvillian(hamiltonian, j_ops)

# create the ansatz model
rbm = nk.models.NDM(alpha=1, beta=1, use_visible_bias=True, use_hidden_bias=True, use_ancilla_bias=True, kernel_init=nn.initializers.normal(stddev=0.1),
                    bias_init=nn.initializers.normal(stddev=0.1), visible_bias_init=nn.initializers.normal(stddev=0.1))

# define the sampler
sampler = nk.sampler.MetropolisLocal(lind.hilbert, n_chains=16)

# create the variational state
v_state = nk.vqs.MCMixedState(sampler, rbm, n_samples=2000, n_samples_diag=1000, n_discard_per_chain=50)
v_state.init_parameters(jax.nn.initializers.normal(stddev=0.001), seed=1)


In [ ]:
# calcualte steady state density matrix with qutip
rho_ss = qt.steadystate(hamiltonian.to_qobj(), [j_op.to_qobj() for j_op in j_ops])

In [ ]:
# calcualte observables with qutip
obx_q = obs_sx.to_qobj()
ony_q = obs_sy.to_qobj()
obz_q = obs_sz.to_qobj()

print("\n", "x-magnetization:", qt.expect(obx_q, rho_ss),"\n", "y-magnetization:", qt.expect(ony_q, rho_ss),"\n", "z-magnetization:", qt.expect(obz_q, rho_ss))


In [ ]:
# calculate observales with netket exact method
rho_nk = nk.exact.steady_state(lind, method='iterative', rho0=v_state.to_matrix())

obsx = obs_sy.to_dense()
obsy = obs_sy.to_dense()
obsz = obs_sz.to_dense()

sx, sy, sz = np.real(np.trace(rho_nk @ obsx)), np.real(np.trace(rho_nk @ obsy)), np.real(np.trace(rho_nk @ obsz))
print("\n", "x-magnetization:", np.round(sx, 5),"\n", "y-magnetization:", np.round(sy,5), "\n", "z-magnetization:",np.round(sz,5))

# Simulation part

In [ ]:
# defining the optimizer
optimizer = nk.optimizer.Sgd(0.01)
sr = nk.optimizer.SR(diag_shift=1e-2, holomorphic=False)

# create the simulation object minimizing the Liouvillian
ss = nk.SteadyState(lind, variational_state=v_state, optimizer=optimizer, preconditioner=sr)

# save the data whicle running
logger= nk.logging.RuntimeLog()

# start the simulation defining the observables measured at each iteration
ss.run(out=logger, n_iter=5000, obs={"Sx": obs_sx, "Sy": obs_sy, "Sz": obs_sz})

# Results 

In [ ]:
# calculate fidelity with qutip matrix and variational state
sigma = v_state.to_qobj()
print(" The fidelity is:", qt.fidelity(rho_ss, sigma))

In [ ]:
# calculate the expectation values based on the current variatonal state

sx_expect = v_state.expect(obs_sx).mean.real
sy_expect = v_state.expect(obs_sy).mean.real
sz_expect = v_state.expect(obs_sz).mean.real
print("\n", "x-magnetization:",sx_expect, "\n", "y-magnetization:",sy_expect, "\n", "z-magnetization:", sz_expect)

In [ ]:
# convert the simulation data from the logger

saved_params = logger.data

x = saved_params["Sx"]["iters"]
y_sx = saved_params["Sx"]["Mean"]
y_sy = saved_params["Sy"]["Mean"]
y_sz = saved_params["Sz"]["Mean"]

LdagL = saved_params["LdagL"]["Mean"]

In [ ]:
# calculate the expectation values based on the chosen number of last iterations

sx_calc = y_sx[-1000:].mean().real
sy_calc = y_sy[-1000:].mean().real
sz_calc = y_sz[-1000:].mean().real
print("\n", "x-magnetization:", sx_calc, "\n", "y-magnetization:", sy_calc, "\n", "z-magnetization:",sz_calc)

In [ ]:
# plot the results

plt.plot(x, LdagL)
plt.title("LdagL")
plt.xlabel('Iteration')
plt.ylabel('<LdagL>')
plt.show()

plt.plot(x, y_sx) 
plt.axhline(y=sx_expect, color='r', linestyle='--')
plt.title("Sx")
plt.xlabel('Iteration')
plt.ylabel('Magnetization X')
plt.show()

plt.plot(x, y_sy)
plt.axhline(y=sy_expect, color='r', linestyle='--')
plt.xlabel('Iteration')
plt.ylabel('Magnetization Y')
plt.title("Sy")
plt.show()

plt.plot(x, y_sz)
plt.axhline(y=sz_expect, color='r', linestyle='--')
plt.xlabel('Iteration')
plt.ylabel('Magnetization Z')
plt.show()

